In [2]:
import csv
import re
import os
import sys
import lxml
from lxml import etree
from lxml import objectify
from xml.etree import ElementTree
from xml.etree.ElementTree import iterparse
from io import StringIO

ns = {'v3':'http://www.loc.gov/mods/v3'}
path = "/Users/Sandy/Documents/Work/DI/Peel/postcards"
fileobject = 'postcards/PC000001.xml'
fileobject2 = 'postcards/PC000035.xml'

In [ ]:
tree1 = etree.parse(fileobject)
root1 = tree1.getroot()
print(root1)
print(len(root1))
for child in root1:
    print(child)

In [ ]:
all_links = tree1.findall('{http://www.loc.gov/mods/v3}note')
all_links[0].text

-----

In [44]:
tree2 = etree.parse(fileobject2)

In [94]:
tree2.xpath('(//v3:name[@type="corporate"]/*/text())[1]', namespaces=ns)

['Standard Photo Co']

-----

In [ ]:
root = etree.parse('postcards/PC000002.xml')

In [ ]:
r = root.xpath('//text()')
print(r)

In [ ]:
#this works

for node in root.xpath('//v3:note/text()', namespaces=ns):
    print(node)

In [ ]:
#this works

for e in root.xpath('//v3:mods', namespaces=ns):
    print(e.xpath('//v3:title/text()',namespaces=ns)[0])
    print(e.xpath('//v3:title/text()',namespaces=ns)[1]) 
    print(e.xpath('//v3:namePart/text()', namespaces=ns)[0])
    print(e.xpath('//v3:namePart/text()', namespaces=ns)[1])
    print(e.xpath('//v3:namePart/text()', namespaces=ns)[2])

In [ ]:
#this works

with open('output1.csv', 'w') as csvfile:
    fieldnames = ['id', 'city']
    writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
    writer.writeheader()
    for e in root.xpath('//v3:mods', namespaces=ns):
        ident = e.xpath('//v3:recordIdentifier/text()',namespaces=ns)[0] 
        city = e.xpath('//v3:city/text()', namespaces=ns)[0]
        writer.writerow({'id': ident, 'city': city})

------

In [ ]:
#this works
with open('postcards/PC000001.xml', 'rt') as f:
    tree = ElementTree.parse(f)
print(tree)

In [ ]:
#this works
for node in tree.iter():
    print(node.tag, node.attrib, node.text)

------

In [ ]:
text = doc.xpath("/titleInfo/title/text()")
print(text)

In [ ]:
print(text)

In [ ]:
doc1 = etree.parse(fileobject)

In [ ]:
#this works
for element in doc1.iter():
    print("%s - %s" % (element.tag, element.text))

In [ ]:
value = line.split("{http://www.loc.gov/mods/v3}").pop()

In [ ]:
def get_namespace(element):
  m = re.match('\{.*\}', element.tag)
  return m.group(0) if m else ''

In [ ]:
namespace = get_namespace(doc1.getroot())
print(namespace)


el2 = tree.findall("{%s}DEAL_LEVEL/{%s}PAID_OFF" %(ns,ns))

In [ ]:
nn = "http://www.loc.gov/mods/v3"
city1 = doc1.findall('{%s}note' %(nn))
print(city1)

In [ ]:
#there's something here
doclib = []
for element in doc1.iter('{http://www.loc.gov/mods/v3}note','{http://www.loc.gov/mods/v3}city' ):
    #doclib.append(element.tag)
    doclib.append(element.text)

In [ ]:
print(doclib)

In [ ]:
doclist=[]
for element in doc1.iter(('//v3:recordIdentifier/text()', namespaces=ns),('//v3:city/text()', namespaces=ns)):
    doclist.append(element.text)

In [ ]:
print(doclist)

-----

This is working

In [ ]:
rootdir = "/Users/Sandy/Documents/Work/DI/Peel/postcards"

In [ ]:
cards = []
for subdir, dirs, files in os.walk(rootdir): 
#iterate through each file        
    for fp in files:
        file = subdir + "/" + fp
        tree = etree.parse(file)
        cards.append(tree)

In [ ]:
print(len(cards))

In [ ]:
lines = []
for card in cards:
    for element in card.iter():
        lines.append(element.text)

In [ ]:
for card in cards:
    r = etree.parse(card)

------

In [22]:
#this works

dirs = os.listdir(path)
parsed = []
for filename in dirs:
    page = os.path.join(path, filename)
    if os.path.isfile(page):
        parsed.append(etree.parse(page))

In [64]:
#this works

with open('output.csv', 'w') as csvfile:
    fieldnames = ['id', 'date', 'city', 'province','country', 'title','url', 'subject1']
    writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
    writer.writeheader()
    for card in parsed:
        for e in card.xpath('//v3:mods', namespaces=ns):
            ident = e.xpath('//v3:recordIdentifier/text()',namespaces=ns) 
            date = e.xpath('//v3:dateIssued/text()', namespaces=ns)
            city = e.xpath('//v3:city/text()', namespaces=ns)
            prov = e.xpath('//v3:province/text()', namespaces=ns)
            country = e.xpath('//v3:country/text()', namespaces=ns)
            title = e.xpath('//v3:title/text()', namespaces=ns)
            url = e.xpath('//v3:url[@usage="primary display"]/text()', namespaces=ns)
            subject1 = e.xpath('//v3:topic/text()', namespaces=ns)
            writer.writerow({'id': ident, 'date':date, 'city': city, 'province':prov, 'country':country, 'title':title, 'url':url, 'subject1':subject1})

In [104]:
#this works

with open('outputRole.csv', 'w') as csvfile:
    fieldnames = ['id', 'nameCorporate','namePersonal', 'role']
    writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
    writer.writeheader()
    for card in parsed:
        for e in card.xpath('//v3:mods', namespaces=ns):
            ident = e.xpath('//v3:recordIdentifier/text()',namespaces=ns) 
            nameCorporate = e.xpath('(//v3:name[@type="corporate"]/*/text())[1]', namespaces=ns)
            namePersonal = e.xpath('(//v3:name[@type="personal"]/*/text())[1]', namespaces=ns)
            role = e.xpath('(//v3:roleTerm/text())[1]', namespaces=ns)
            writer.writerow({'id': ident, 'nameCorporate':nameCorporate, 'namePersonal':namePersonal, 'role':role})